<a href="https://colab.research.google.com/github/AkankshaB123/leet-code/blob/main/Caller_History_Normalisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1) Why would have happened if this normalisation wouldn't exist?

*   Subjective definition: *Normalization is structuring data so each row represents one atomic fact or event, which makes time-based and user-based analysis correct and simple*  
*   Because real analytics always starts with normalization: Logs, Clickstreams, Payments.

2) How can this be solved in Customer-to-Customer business? eg. eBay, Uber, Meta.

*   UNION ALL - Buyer:Seller convert into user_id | role; Rider:Driver convert into user_id | role

## **3) Practical explanation below**








# **Call History - Not Normalised**

In [4]:
import sqlite3

conn = sqlite3.connect(":memory:")
cur = conn.cursor()

cur.execute("""
CREATE TABLE calls (
    call_id INTEGER,
    caller TEXT,
    recipient TEXT,
    call_time INTEGER
)
""")

cur.executemany("""
INSERT INTO calls VALUES (?, ?, ?, ?)
""", [
    (1, 'A', 'B', 1),
    (2, 'B', 'C', 2),
    (3, 'A', 'C', 3)
])

conn.commit()

# **❌ WRONG SQL (no normalization)**

In [5]:
#Question: users with consecutive calls
#Mistake: only tracking callers
cur.execute("""
WITH ordered_calls AS (
    SELECT
        caller AS user,
        call_time,
        LAG(call_time) OVER (
            PARTITION BY caller
            ORDER BY call_time
        ) AS prev_time
    FROM calls
)
SELECT DISTINCT user
FROM ordered_calls
WHERE call_time - prev_time = 1;
""")

cur.fetchall()


[]

## **✅ CORRECT SQL (normalized with UNION ALL)**

In [6]:
cur.execute("""
WITH user_events AS (
    SELECT caller AS user, call_time FROM calls
    UNION ALL
    SELECT recipient AS user, call_time FROM calls
),
ordered_events AS (
    SELECT
        user,
        call_time,
        LAG(call_time) OVER (
            PARTITION BY user
            ORDER BY call_time
        ) AS prev_time
    FROM user_events
)
SELECT DISTINCT user
FROM ordered_events
WHERE call_time - prev_time = 1;
""")

cur.fetchall()


[('B',), ('C',)]

In [8]:
# call_id | caller | recipient | call_time
# ----------------------------------------
# 1       | A      | B         | 1
# 2       | B      | C         | 2
# 3       | A      | C         | 3

# | user | call_time |
# | ---- | --------- |
# | A    | 1         |
# | B    | 1         |
# | B    | 2         |
# | C    | 2         |
# | A    | 3         |
# | C    | 3         |

# User A: 1-3 = 2
# User B: 1-2 = 1
# User C: 2-3 = 1